In [1]:
# Cell 1: Imports and Warnings
import numpy as np
import pandas as pd
from scipy.integrate import simps, cumulative_trapezoid
from scipy.constants import G as Const_G # Gravitational constant (SI units)
import os
import warnings
from joblib import Parallel, delayed # For parallelization
import sys # For flushing output

# Suppress RuntimeWarnings
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in true_divide')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='divide by zero encountered in true_divide')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in power')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='divide by zero encountered in power')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in multiply')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in subtract')

In [2]:
# Cell 2: Helper Functions

def _derivative(y, x):
    """Calculate the derivative using np.gradient."""
    return np.gradient(y, x, edge_order=2)

def _integrate(y, x):
    """Integrate using Simpson's rule."""
    return simps(y, x)

def _cumulative_integrate(y, x, initial=0):
    """Cumulative integration using trapezoidal rule."""
    return cumulative_trapezoid(y, x, initial=initial)

def _get_k_ang(l_val):
    """Calculate angular wavenumber k_l."""
    return np.sqrt(l_val * (l_val + 1)) if l_val > 0 else 1e-9

def _load_frequencies_from_res1(res1_path, sigma_char, target_map, nl_tuples_to_consider):
    """
    Helper function to load frequencies from a res1.txt file.
    nl_tuples_to_consider is a list of (n,l) tuples.
    """
    print(f"Attempting to load {sigma_char}-mode frequencies from: {res1_path}")
    nl_set_to_consider = set(nl_tuples_to_consider)
    if not nl_set_to_consider:
        print(f"  No {sigma_char}-modes specified for loading frequencies.")
        return

    if os.path.exists(res1_path):
        try:
            data_freq = np.loadtxt(res1_path, comments='#')
            if data_freq.ndim == 1: data_freq = data_freq.reshape(1, -1)
            
            loaded_count = 0
            for row in data_freq:
                n_file, l_file = int(row[0]), int(row[1])
                if (n_file, l_file) not in nl_set_to_consider:
                    continue
                
                # Assuming format: n l T Q f(mHz)
                # Or format: n l T f(mHz)
                try:
                    freq_mhz = float(row[3]) # Standard format
                except IndexError:
                    print(f"  Warning: Row format unexpected for {n_file},{l_file}. Skipping.")
                    continue
                    
                omega_sq_val = (freq_mhz * 1e-3 * 2 * np.pi)**2
                if omega_sq_val > 1e-20:
                    target_map[(sigma_char, n_file, l_file)] = omega_sq_val
                    loaded_count += 1
                else:
                    print(f"  Warning: Skipping zero/negative omega_sq ({omega_sq_val:.2e}) for ({sigma_char},{n_file},{l_file})")
            print(f"  Successfully loaded {loaded_count} relevant {sigma_char}-mode frequencies.")
        except Exception as e:
            print(f"  Error parsing {res1_path}: {e}. Frequencies for {sigma_char}-modes might be missing.")
    else:
        print(f"  Warning: Eigenfrequency file '{res1_path}' for {sigma_char}-modes not found.")

In [3]:
# Cell 3: The Main Simplified S2 Calculator Class

class SimplifiedS2Calculator:
    """
    A simplified calculator for spherically symmetric (s=0) perturbations
    coupling Spheroidal l=2 modes with other Spheroidal l=2 modes.
    
    Provides functions to calculate V_mat - omega^2 * T_mat coefficients.
    """

    def __init__(self, 
                 all_s2_n_values, 
                 model_file, 
                 eigenfunction_folder_spher,
                 omega_k2_map,
                 G_const=0.0 # IMPORTANT: Set to 0.0 to match original code's logic
                ):
        """
        Initializes the calculator by loading the model and all specified
        S, l=2 eigenfunctions.

        Args:
            all_s2_n_values (list): List of n values for l=2 [0, 1, 2, ...].
            model_file (str): Path to the background model file.
            eigenfunction_folder_spher (str): Path to the folder with S-mode eigenfunctions.
            omega_k2_map (dict): Pre-loaded map of {('S', n, 2): omega_k_sq}.
            G_const (float): Gravitational constant. Per original code, this is 0.0.
                             Set to Const_G (from scipy) for physical calculations.
        """
        
        print("--- Initializing SimplifiedS2Calculator ---")
        
        # --- Store constants and parameters ---
        self.l = 2
        self.k_ang = _get_k_ang(self.l)
        self.G = G_const # Set G (0.0 based on original code)
        if self.G == 0.0:
            print("Warning: Gravitational constant G is set to 0.0 (matching original code logic).")
            print("         This means g_model, p_val, and V_rho kernel will be 0.")
        
        # --- B-factors and Angular Factor for S=0, L=2, SS coupling ---
        # These are derived from the original code's logic for this specific case
        self.B0_PLUS_S0L2 = 1.0 / np.sqrt(5.0)
        self.B1_PLUS_S0L2 = 6.0 / np.sqrt(5.0)
        self.B2_PLUS_S0L2 = 24.0 / np.sqrt(5.0)
        self.ANGULAR_FACTOR_S0L2 = np.sqrt(5.0) # Combined (Ang. Factor * Pert. Factor)

        # --- Load Background Model ---
        print(f"Loading background model from: {model_file}")
        try:
            self._load_background_model(model_file)
            self.R_surface = self.r_model[-1]
            print(f"Model loaded: {len(self.r_model)} radial points, R = {self.R_surface / 1e3:.1f} km")
        except Exception as e:
            print(f"Error loading model file: {e}")
            raise

        # --- Validate and Prepare Eigenfunctions ---
        self.eigen_data = {}
        self.omega_k2_map_instance = {}
        
        # Filter k_tuples to only those with valid omega
        self.k_values_with_data = []
        for n_val in sorted(list(set(all_s2_n_values))):
            k_tuple = ('S', n_val, self.l)
            if k_tuple not in omega_k2_map:
                print(f"  Skipping {k_tuple}: Frequency not found in provided omega_k2_map.")
                continue
            if omega_k2_map[k_tuple] <= 1e-20:
                 print(f"  Skipping {k_tuple}: Non-positive omega_k^2 ({omega_k2_map[k_tuple]:.2e}).")
                 continue
            
            try:
                # Check file existence *before* adding to list
                self._check_eigenfunction_file_exists(k_tuple, eigenfunction_folder_spher)
                self.k_values_with_data.append(k_tuple)
                self.omega_k2_map_instance[k_tuple] = omega_k2_map[k_tuple]
            except FileNotFoundError as e:
                print(f"  Skipping {k_tuple}: Eigenfunction file not found ({e}).")

        self.num_k_modes = len(self.k_values_with_data)
        if self.num_k_modes == 0:
            print("Warning: No valid modes found with both frequency and eigenfunction data.")
            return

        print(f"\nFound {self.num_k_modes} valid (S, n, 2) modes to process.")
        
        # --- Process Eigenfunctions ---
        self._prepare_all_eigenfunctions(eigenfunction_folder_spher)
        print("--- Initialization Complete ---")

    
    def _load_background_model(self, modname):
        model = np.loadtxt(modname, skiprows=0)
        self.r_model = model[:, 0]
        self.rho_model = model[:, 1]
        self.vp_model = model[:, 2]
        self.vs_model = model[:, 3]
        self.mu_model = self.rho_model * self.vs_model**2
        self.kappa_model = self.rho_model * (self.vp_model**2 - (4.0/3.0) * self.vs_model**2)
        self.kappa_model[self.kappa_model < 0] = 0
        self.r_stable = np.maximum(self.r_model, 1e-10)
        
        # g_model calculation depends on self.G
        mass_integrand = self.rho_model * self.r_model**2
        mass_r = 4 * np.pi * _cumulative_integrate(mass_integrand, self.r_model)
        self.g_model = np.divide(-self.G * mass_r, self.r_stable**2, 
                                 out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        if len(self.g_model)>0: self.g_model[0] = 0.0

    def _get_eigen_fname_path(self, k_tuple, eigenfunction_folder_spher):
        sigma_char, n_val, l_val = k_tuple
        n_str_formatted = f"{n_val:07d}"
        l_str_formatted = f"{l_val:07d}"
        prefix = "S."
        filename_path = os.path.join(eigenfunction_folder_spher, 
                                     f"{prefix}{n_str_formatted}.{l_str_formatted}.ASC")
        if not os.path.exists(filename_path):
             raise FileNotFoundError(filename_path)
        return filename_path

    def _check_eigenfunction_file_exists(self, k_tuple, eigenfunction_folder_spher):
        self._get_eigen_fname_path(k_tuple, eigenfunction_folder_spher)

    def _load_single_eigenfunction(self, k_tuple, eigenfunction_folder_spher):
        sigma_char, n_val, l_val = k_tuple
        filename = self._get_eigen_fname_path(k_tuple, eigenfunction_folder_spher)
        
        df = pd.read_csv(filename, skiprows=1, header=None, delim_whitespace=True)
        
        U_M_raw = np.array(df[1], dtype=float)[::-1]
        V_M_raw = np.array(df[3], dtype=float)[::-1]

        if len(U_M_raw) != len(self.r_model) or len(V_M_raw) != len(self.r_model):
            raise ValueError(f"Length mismatch for S-mode {k_tuple}. File: {filename}.")

        # Note: l=0 logic removed as we are fixed to l=2
        norm_integrand = self.rho_model * \
                         (U_M_raw**2 + self.k_ang**2 * V_M_raw**2) * \
                         self.r_model**2
        
        norm_factor_val_sq = _integrate(norm_integrand, self.r_model)

        if norm_factor_val_sq < 1e-20:
            print(f"Warning: S-mode Normalization factor squared for {k_tuple} is small ({norm_factor_val_sq}).")
            norm_factor_sqrt = 1e10
        else:
            norm_factor_sqrt = np.sqrt(norm_factor_val_sq)
        
        U_latex = U_M_raw / norm_factor_sqrt
        V_latex = (self.k_ang * V_M_raw) / norm_factor_sqrt
        
        return {'U_latex': U_latex, 'V_latex': V_latex}

    def _process_single_eigenfunction_job(self, k_tuple_job, eigenfunction_folder_spher_job):
        """Processes one eigenfunction, calculating all derivatives and fields."""
        sigma_char, n_val, l_val = k_tuple_job
        
        raw_efuncs_latex = self._load_single_eigenfunction(k_tuple_job, 
                                                           eigenfunction_folder_spher_job)
        data = {'sigma': sigma_char, 'n': n_val, 'l': l_val}
        
        data['u'] = raw_efuncs_latex['U_latex']
        data['v_div_k'] = np.divide(raw_efuncs_latex['V_latex'], self.k_ang, 
                                    out=np.zeros_like(raw_efuncs_latex['V_latex']), 
                                    where=self.k_ang!=0)
        
        data['dot_u'] = _derivative(data['u'], self.r_model)
        data['dot_v_div_k'] = _derivative(data['v_div_k'], self.r_model)
        
        data['f_val'] = np.divide(2*data['u'] - self.k_ang**2 * data['v_div_k'], self.r_stable, 
                                  out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        
        term_v_div_k_r = np.divide(data['v_div_k'], self.r_stable, 
                                   out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        term_u_r = np.divide(data['u'], self.r_stable, 
                             out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        data['x_val'] = data['dot_v_div_k'] - term_v_div_k_r + term_u_r
        
        # --- P_val calculation (depends on self.G) ---
        V_for_P_calc = raw_efuncs_latex['V_latex']
        
        integrand1_p = self.rho_model * (self.l * data['u'] + V_for_P_calc) * self.r_stable**(self.l + 1)
        integrand2_p = self.rho_model * (-(self.l + 1) * data['u'] + V_for_P_calc) * \
                       np.power(self.r_stable, -self.l, 
                                out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        
        int1_p = _cumulative_integrate(integrand1_p, self.r_model)
        int2_total_p = _integrate(integrand2_p, self.r_model)
        int2_cumulative_p = _cumulative_integrate(integrand2_p, self.r_model)
        int2_rev_p = int2_total_p - int2_cumulative_p
        
        P_term1 = np.divide(int1_p, np.power(self.r_stable, self.l + 1, 
                                            out=np.ones_like(self.r_model), where=self.r_stable!=0),
                            out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        P_term2 = int2_rev_p * np.power(self.r_stable, self.l, 
                                        out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        
        data['p_val'] = -4 * np.pi * self.G / (2*self.l + 1) * (P_term1 + P_term2) # Use self.G
        if len(data['p_val']) > 0: data['p_val'][0] = 0.0
        data['dot_p'] = _derivative(data['p_val'], self.r_model)

        return k_tuple_job, data

    def _prepare_all_eigenfunctions(self, eigenfunction_folder_spher):
        print("Preparing all eigenfunction derivatives (S,l=2) (parallelized)...")
        
        tasks = [delayed(self._process_single_eigenfunction_job)(k_tuple, eigenfunction_folder_spher)
                 for k_tuple in self.k_values_with_data]
        
        # Use n_jobs=-2 (all cores but one) or a fixed number
        results = Parallel(n_jobs=-2, backend='loky', verbose=5)(tasks) 

        for k_tuple_res, data_res in results:
            self.eigen_data[k_tuple_res] = data_res
        
        print(f"\nAll {len(self.k_values_with_data)} (S,l=2) eigenfunctions processed.")

    def _get_kernels_s0_l2_ss(self, data_k1, data_k2):
        """
        Calculates the SS kernels for s_pert=0, l=2, simplified logic.
        """
        kernels = {}
        
        # Get pre-calculated B-factors
        B0p = self.B0_PLUS_S0L2
        B1p = self.B1_PLUS_S0L2
        B2p = self.B2_PLUS_S0L2
        
        # Get model data
        r_stable = self.r_stable
        rho_model = self.rho_model
        g_model = self.g_model
        kappa_model = self.kappa_model
        mu_model = self.mu_model
        
        # Get eigenfunction data
        u1,v1dk,du1,f1,x1,p1,dp1 = (data_k1['u'], data_k1['v_div_k'], data_k1['dot_u'],
                                   data_k1['f_val'], data_k1['x_val'], data_k1['p_val'],
                                   data_k1['dot_p'])
        u2,v2dk,du2,f2,x2,p2,dp2 = (data_k2['u'], data_k2['v_div_k'], data_k2['dot_u'],
                                   data_k2['f_val'], data_k2['x_val'], data_k2['p_val'],
                                   data_k2['dot_p'])

        # --- T_rho Kernel ---
        kernels['T_rho_SS'] = u1*u2*B0p + v1dk*v2dk*B1p
        
        # --- V_kappa Kernel ---
        kernels['V_kappa_SS'] = (du1+f1)*(du2+f2)*B0p
        
        # --- V_mu Kernel ---
        V_mu_SS_term1 = (1./3.)*(2*du1-f1)*(2*du2-f2)*B0p
        V_mu_SS_term2 = x1*x2*B1p
        V_mu_SS_term3 = np.divide(v1dk*v2dk, r_stable**2, 
                                  out=np.zeros_like(rho_model), where=(r_stable!=0))*B2p
        kernels['V_mu_SS'] = V_mu_SS_term1 + V_mu_SS_term2 + V_mu_SS_term3

        # --- V_rho Kernel (depends on G) ---
        V_rho_SS_term1 = (u1*dp2 + dp1*u2 - 0.5*g_model * \
                         (np.divide(4*u1*u2,r_stable,out=np.zeros_like(rho_model),where=r_stable!=0) + f1*u2 + u1*f2) + \
                         8*np.pi*self.G*rho_model*u1*u2)*B0p
        V_rho_SS_term2 = np.divide((p1*v2dk + v1dk*p2) + 0.5*g_model*(u1*v2dk + v1dk*u2), \
                                 r_stable, out=np.zeros_like(rho_model), where=r_stable!=0)*B1p
        kernels['V_rho_SS'] = V_rho_SS_term1 + V_rho_SS_term2
        
        # --- V_d Kernel (simplified for s=0) ---
        Vd_SS_term_bg = -kappa_model*kernels['V_kappa_SS'] \
                        -mu_model*kernels['V_mu_SS'] \
                        -rho_model*kernels['V_rho_SS']
        
        Vd_SS_kappa_deriv = kappa_model*( \
                                      (2*du1*du2 + du1*f2 + f1*du2)*B0p \
                                      # B_l2_l1_s_1_plus term is 0 for s=0, l=2
                                      # B_l1_l2_s_1_plus term is 0 for s=0, l=2
                                      )
        Vd_SS_mu_deriv = mu_model*( \
                                    (2./3.)*(4*du1*du2 - du1*f2 - f1*du2)*B0p \
                                    + (data_k1['dot_v_div_k']*x2 + x1*data_k2['dot_v_div_k'])*B1p \
                                    # B_l2_l1_s_1_plus term is 0 for s=0, l=2
                                    # B_l1_l2_s_1_plus term is 0 for s=0, l=2
                                    )
        kernels['V_d_SS'] = Vd_SS_term_bg + Vd_SS_kappa_deriv + Vd_SS_mu_deriv
        
        # --- T_d Kernel ---
        kernels['T_d_SS'] = -rho_model * kernels['T_rho_SS']
        
        return kernels

    def _get_jump_value(self, r_interface, kernel_array, 
                        surface_proximity_threshold=500.0, jump_half_width=50.0):
        """Calculates the jump [f] = f(r_i+) - f(r_i-) for a kernel array."""
        
        # Check if it's a surface perturbation
        if abs(r_interface - self.R_surface) < surface_proximity_threshold:
            # Surface jump: [f] = f(R+) - f(R-) = 0.0 - f(R-)
            jump_val = 0.0 - kernel_array[-1]
        else:
            # Internal interface jump
            jump_val = 0.0
            idx_below_arr = np.where(self.r_model < (r_interface - jump_half_width))[0]
            idx_above_arr = np.where(self.r_model >= (r_interface + jump_half_width))[0]
            
            if len(idx_below_arr) > 0 and len(idx_above_arr) > 0:
                idx_b, idx_a = idx_below_arr[-1], idx_above_arr[0]
                if idx_b < idx_a and idx_a < len(self.r_model):
                    # [f] = f(r_i+) - f(r_i-)
                    jump_val = kernel_array[idx_a] - kernel_array[idx_b]
            else:
                print(f"Warning: Could not find points above/below interface r={r_interface}."
                      f" Jump value will be 0.")
        
        return jump_val

    # --- Public Functions (Requirement 2 & 3) ---

    def calculate_coupling_term(self, n1, n2, pert_type, pert_profile, omega_n2_sq):
        """
        Calculates V_mat(n1, n2) - omega_n2^2 * T_mat(n1, n2) for a specific 
        perturbation type and profile.

        Args:
            n1 (int): Radial quantum number of mode 1 (bra).
            n2 (int): Radial quantum number of mode 2 (ket).
            pert_type (str): 'delta_rho', 'delta_mu', 'delta_kappa', or 'delta_d'.
            pert_profile (tuple or float): 
                - For 'delta_rho', 'mu', 'kappa': (r_min, r_max) tuple.
                - For 'delta_d': r_interface (float).
            omega_n2_sq (float): Unperturbed omega^2 for mode n2.

        Returns:
            float: The value of V_mat - omega_n2^2 * T_mat.
        """
        k1_tuple = ('S', n1, self.l)
        k2_tuple = ('S', n2, self.l)

        # Check if we have data for these modes
        if k1_tuple not in self.eigen_data or k2_tuple not in self.eigen_data:
            print(f"Error: Eigenfunction data not found for {k1_tuple} or {k2_tuple}.")
            print(f"Available n values: {[k[1] for k in self.k_values_with_data]}")
            return np.nan

        data_k1 = self.eigen_data[k1_tuple]
        data_k2 = self.eigen_data[k2_tuple]

        # Get the s=0, l=2, SS kernels
        kernels_s0 = self._get_kernels_s0_l2_ss(data_k1, data_k2)

        V_mat = 0.0
        T_mat = 0.0
        r = self.r_model
        
        if pert_type in ['delta_rho', 'delta_mu', 'delta_kappa']:
            r_min, r_max = pert_profile
            # Create perturbation profile P(r) = 1 inside range, 0 outside
            P_r = np.zeros_like(r)
            mask = (r >= r_min) & (r <= r_max)
            P_r[mask] = 1.0

            if pert_type == 'delta_rho':
                # V_rho_SS kernel is 0 if G=0
                kernel_V = kernels_s0['V_rho_SS'] 
                kernel_T = kernels_s0['T_rho_SS']
                integrand_V = P_r * kernel_V * r**2
                integrand_T = P_r * kernel_T * r**2
                V_mat = self.ANGULAR_FACTOR_S0L2 * _integrate(integrand_V, r)
                T_mat = self.ANGULAR_FACTOR_S0L2 * _integrate(integrand_T, r)
                
            elif pert_type == 'delta_mu':
                kernel_V = kernels_s0['V_mu_SS']
                kernel_T = 0.0 # No T_mu kernel
                integrand_V = P_r * kernel_V * r**2
                V_mat = self.ANGULAR_FACTOR_S0L2 * _integrate(integrand_V, r)
                T_mat = 0.0

            elif pert_type == 'delta_kappa':
                kernel_V = kernels_s0['V_kappa_SS']
                kernel_T = 0.0 # No T_kappa kernel
                integrand_V = P_r * kernel_V * r**2
                V_mat = self.ANGULAR_FACTOR_S0L2 * _integrate(integrand_V, r)
                T_mat = 0.0
        
        elif pert_type == 'delta_d':
            r_interface = pert_profile
            d_val = 1.0 # We are calculating the coefficient

            kernel_V_d = kernels_s0['V_d_SS']
            kernel_T_d = kernels_s0['T_d_SS']

            jump_Vd_val = self._get_jump_value(r_interface, kernel_V_d)
            jump_Td_val = self._get_jump_value(r_interface, kernel_T_d)
            
            V_mat = self.ANGULAR_FACTOR_S0L2 * (r_interface**2 * d_val * jump_Vd_val)
            T_mat = self.ANGULAR_FACTOR_S0L2 * (r_interface**2 * d_val * jump_Td_val)
            
        else:
            print(f"Error: Unknown pert_type '{pert_type}'")
            return np.nan

        return V_mat - omega_n2_sq * T_mat

    def calculate_diagonal_term(self, n, pert_type, pert_profile, omega_n_sq):
        """
        Calculates the diagonal term V_mat(n, n) - omega_n^2 * T_mat(n, n).
        This is a convenience wrapper for calculate_coupling_term.
        """
        return self.calculate_coupling_term(n, n, pert_type, pert_profile, omega_n_sq)

In [4]:
# Cell 4: Fisher Matrix Calculator Class

import numpy as np
from numpy.linalg import inv, pinv
import warnings
# 'Parallel' and 'delayed' are imported in Cell 1
# from joblib import Parallel, delayed 

class FisherMatrixCalculator:
    """
    Calculates Fisher and Posterior Covariance matrices based on the 
    SimplifiedS2Calculator results.
    
    [MODIFIED] Jacobian calculations are parallelized using joblib.
    """
    
    def __init__(self, calculator: SimplifiedS2Calculator, r_obs_meters: float):
        """
        Initializes the Fisher matrix calculator.

        Args:
            calculator: An initialized instance of SimplifiedS2Calculator.
            r_obs_meters: The radial position (in meters) at which amplitudes
                          (U, V) are 'observed'.
        """
        if calculator.num_k_modes == 0:
            raise ValueError("Calculator has no valid modes loaded.")
            
        print(f"--- Initializing FisherMatrixCalculator ---")
        self.calculator = calculator
        self.r_model = calculator.r_model
        
        # Find the index closest to the observation radius
        self.r_obs_idx = np.argmin(np.abs(self.r_model - r_obs_meters))
        self.r_obs_actual = self.r_model[self.r_obs_idx]
        print(f"Observation radius set to r = {self.r_obs_actual/1e3:.1f} km "
              f"(requested {r_obs_meters/1e3:.1f} km)")
        
        # Store all k_tuples and their omega_sq
        self.all_k_tuples = sorted(calculator.k_values_with_data, key=lambda k: k[1])
        self.omega_k2_map = calculator.omega_k2_map_instance
        
        # --- Pre-calculate U_k and V_k at r_obs AND I_k integral ---
        print("Pre-calculating amplitudes U_k, V_k at observation radius...")
        self.U_at_r_obs = {}
        self.V_at_r_obs = {}
        
        # --- [MODIFICATION (from previous round)] Pre-calculate I_k integral ---
        print(f"Pre-calculating I_k integrals (Integral[ d(mu)/dr * [U_k + C*V_k] * r^2 ] dr)...")
        self.I_k_integral = {}
        mu_model = self.calculator.mu_model
        r_model = self.calculator.r_model
        # Use np.gradient (from numpy) for derivative
        dot_mu = np.gradient(mu_model, r_model, edge_order=2) 
        C_factor = 3.0 / np.sqrt(6.0)
        r_sq = r_model**2
        # --- [END MODIFICATION] ---
        
        for k_tuple in self.all_k_tuples:
            data = self.calculator.eigen_data[k_tuple]
            # U_k is 'u'
            u_array = data['u']
            # V_k is 'v_div_k' * k_ang (as per _load_single_eigenfunction)
            v_array = data['v_div_k'] * self.calculator.k_ang
            
            self.U_at_r_obs[k_tuple] = u_array[self.r_obs_idx]
            self.V_at_r_obs[k_tuple] = v_array[self.r_obs_idx]
            
            # --- [MODIFICATION (from previous round)] Calculate and store I_k integral ---
            # We assume 'simps' is in the global scope (imported in Cell 1)
            integrand_I_k = dot_mu * (u_array + C_factor * v_array) * r_sq
            self.I_k_integral[k_tuple] = simps(integrand_I_k, r_model)
            # --- [END MODIFICATION] ---
            
        self.k_ang = self.calculator.k_ang # Store for convenience
        print(f"--- FisherMatrixCalculator Initialized ({len(self.all_k_tuples)} modes) ---\n")

    def _get_valid_n_list(self, n_range):
        """Filters a given n_range against the modes actually loaded."""
        valid_n = []
        loaded_n_values = {k[1] for k in self.all_k_tuples}
        for n in n_range:
            if n in loaded_n_values:
                valid_n.append(n)
            else:
                print(f"Warning: n={n} requested for observable, but not loaded. Skipping.")
        return sorted(list(set(valid_n)))

    # --- [MODIFICATION] Parallel Jacobian Helper Methods ---

    def _compute_jacobian_F1_column(self, i, n1_list, model_params_m_i):
        """Computes the i-th column of the G1 Jacobian matrix (for joblib)."""
        pert_type, pert_profile = model_params_m_i[i]
        num_n1 = len(n1_list)
        column = np.zeros(num_n1)

        for j in range(num_n1): # Loop over observables n1
            n1 = n1_list[j]
            k1_tuple = ('S', n1, self.calculator.l)
            omega_n1_sq = self.omega_k2_map[k1_tuple]
            
            diag_term = self.calculator.calculate_diagonal_term(
                n1, pert_type, pert_profile, omega_n1_sq
            )
            column[j] = diag_term / omega_n1_sq
        
        return (i, column)

    def _compute_jacobian_F2_column(self, i, n2_list, model_params_m_i):
        """Computes the i-th column of the G2 Jacobian matrix (for joblib)."""
        pert_type, pert_profile = model_params_m_i[i]
        num_n2 = len(n2_list)
        column = np.zeros(num_n2)
        
        for j in range(num_n2): # Loop over observables n2
            n2 = n2_list[j]
            k_n2_tuple = ('S', n2, self.calculator.l)
            omega_n2_sq = self.omega_k2_map[k_n2_tuple]
            
            U_n2_obs = self.U_at_r_obs[k_n2_tuple]
            I_n2_integral = self.I_k_integral[k_n2_tuple]
            
            if abs(U_n2_obs) < 1e-25 or abs(I_n2_integral) < 1e-25:
                # print(f"Warning: U_n(r_obs) or I_n is near-zero for n={n2}. G2(n={n2}, m={i}) set to 0.")
                column[j] = 0.0
                continue

            sum_term = 0.0
            for k_tuple in self.all_k_tuples:
                if k_tuple == k_n2_tuple: continue
                    
                k_n_val = k_tuple[1]
                omega_k_sq = self.omega_k2_map[k_tuple]
                freq_denom = omega_n2_sq - omega_k_sq
                if abs(freq_denom) < 1e-20: continue
                    
                U_k_obs = self.U_at_r_obs[k_tuple]
                I_k_integral_k = self.I_k_integral[k_tuple] # Renamed to avoid confusion
                
                ratio_U = U_k_obs / U_n2_obs
                ratio_I = I_k_integral_k / I_n2_integral
                amplitude_ratio = ratio_U + ratio_I
                
                coupling_term = self.calculator.calculate_coupling_term(
                    k_n_val, n2, pert_type, pert_profile, omega_n2_sq
                )
                sum_term += (1.0 / freq_denom) * amplitude_ratio * coupling_term
            
            column[j] = sum_term
        
        return (i, column)

    def _compute_jacobian_F3_column(self, i, n3_list, model_params_m_i):
        """Computes the i-th column of the G3 Jacobian matrix (for joblib)."""
        pert_type, pert_profile = model_params_m_i[i]
        num_n3 = len(n3_list)
        column = np.zeros(num_n3)
        
        for j in range(num_n3): # Loop over observables n3
            n3 = n3_list[j]
            k_n3_tuple = ('S', n3, self.calculator.l)
            omega_n3_sq = self.omega_k2_map[k_n3_tuple]
            
            V_n3_obs = self.V_at_r_obs[k_n3_tuple]
            I_n3_integral = self.I_k_integral[k_n3_tuple]
            
            if abs(V_n3_obs) < 1e-25 or abs(I_n3_integral) < 1e-25:
                # print(f"Warning: V_n(r_obs) or I_n is near-zero for n={n3}. G3(n={n3}, m={i}) set to 0.")
                column[j] = 0.0
                continue

            sum_term = 0.0
            for k_tuple in self.all_k_tuples:
                if k_tuple == k_n3_tuple: continue
                    
                k_n_val = k_tuple[1]
                omega_k_sq = self.omega_k2_map[k_tuple]
                freq_denom = omega_n3_sq - omega_k_sq
                if abs(freq_denom) < 1e-20: continue
                    
                V_k_obs = self.V_at_r_obs[k_tuple]
                I_k_integral_k = self.I_k_integral[k_tuple] # Renamed
                
                ratio_V = V_k_obs / V_n3_obs
                ratio_I = I_k_integral_k / I_n3_integral
                amplitude_ratio = ratio_V + ratio_I
                
                coupling_term = self.calculator.calculate_coupling_term(
                    k_n_val, n3, pert_type, pert_profile, omega_n3_sq
                )
                sum_term += (1.0 / freq_denom) * amplitude_ratio * coupling_term
            
            column[j] = sum_term
            
        return (i, column)
    
    # --- [MODIFICATION] Main Jacobian Build Methods (Now use Parallel) ---

    def _build_jacobian_F1(self, n1_list, model_params_m_i):
        """Builds G1 (Jacobian for F1 - Frequencies) in parallel."""
        num_n1 = len(n1_list)
        num_i = len(model_params_m_i)
        G1 = np.zeros((num_n1, num_i))
        
        if num_n1 == 0:
            return G1
        
        print(f"Building Jacobian G1 (Frequencies) parallelized over {num_i} parameters...")
        
        tasks = [delayed(self._compute_jacobian_F1_column)(i, n1_list, model_params_m_i)
                 for i in range(num_i)]
        
        # Use n_jobs=-1 for all cores
        results = Parallel(n_jobs=-1, verbose=5, backend='loky')(tasks)
        
        for i, column in results:
            G1[:, i] = column
            
        return G1

    def _build_jacobian_F2(self, n2_list, model_params_m_i):
        """Builds G2 (Jacobian for F2 - U-Amplitude) in parallel."""
        num_n2 = len(n2_list)
        num_i = len(model_params_m_i)
        G2 = np.zeros((num_n2, num_i))
        
        if num_n2 == 0:
            return G2
            
        print(f"Building Jacobian G2 (U-Amplitude) parallelized over {num_i} parameters...")
        
        tasks = [delayed(self._compute_jacobian_F2_column)(i, n2_list, model_params_m_i)
                 for i in range(num_i)]
        
        results = Parallel(n_jobs=-1, verbose=5, backend='loky')(tasks)
        
        for i, column in results:
            G2[:, i] = column
        
        return G2

    def _build_jacobian_F3(self, n3_list, model_params_m_i):
        """Builds G3 (Jacobian for F3 - V-Amplitude) in parallel."""
        num_n3 = len(n3_list)
        num_i = len(model_params_m_i)
        G3 = np.zeros((num_n3, num_i))
        
        if num_n3 == 0:
            return G3
            
        print(f"Building Jacobian G3 (V-Amplitude) parallelized over {num_i} parameters...")
        
        tasks = [delayed(self._compute_jacobian_F3_column)(i, n3_list, model_params_m_i)
                 for i in range(num_i)]
        
        results = Parallel(n_jobs=-1, verbose=5, backend='loky')(tasks)
        
        for i, column in results:
            G3[:, i] = column
            
        return G3
    
    # --- [NO CHANGE] Covariance and Fisher Calculation Logic ---
    
    def _build_cov_matrix_inv(self, num_obs, sigma_val):
        """Builds a diagonal C_inv = (1/sigma^2) * I"""
        if num_obs == 0 or sigma_val <= 0:
            return None # No contribution
        
        # C = sigma^2 * I  =>  C_inv = (1/sigma^2) * I
        C_inv = np.diag(np.full(num_obs, 1.0 / (sigma_val**2)))
        return C_inv

    def _calculate_fisher_matrix(self, G, C_inv, num_i):
        """Calculates I = G.T @ C_inv @ G"""
        if C_inv is None or G.shape[0] == 0:
            # No observables, so Fisher matrix is zero
            return np.zeros((num_i, num_i))
        
        try:
            # I = G.T @ C_inv @ G
            I = G.T @ C_inv @ G
            return I
        except ValueError as e:
            print(f"Error in Fisher matrix calculation (G.T @ C_inv @ G): {e}")
            print(f"G shape: {G.shape}, C_inv shape: {C_inv.shape}")
            return np.zeros((num_i, num_i))

    def calculate_posterior_covariance(self, 
                                     model_params_m_i,
                                     n_range_F1, 
                                     n_range_F2, 
                                     n_range_F3,
                                     sigma_F1, 
                                     sigma_F2, 
                                     sigma_F3,
                                     C_prior,
                                     combinations_dict):
        """
        Calculates the full posterior covariance matrix for various combinations
        of observables.
        """
        
        num_i = len(model_params_m_i)
        if num_i == 0:
            print("Error: No model parameters (m_i) defined.")
            return {}
            
        print(f"Calculating for {num_i} model parameters (m_i).")
        
        # --- Validate observable lists ---
        n1_list = self._get_valid_n_list(n_range_F1)
        n2_list = self._get_valid_n_list(n_range_F2)
        n3_list = self._get_valid_n_list(n_range_F3)
        
        num_n1 = len(n1_list)
        num_n2 = len(n2_list)
        num_n3 = len(n3_list)
        print(f"Observables: {num_n1} (F1-Freq), {num_n2} (F2-U_Amp), {num_n3} (F3-V_Amp)")

        # --- Calculate Prior Information Matrix ---
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            I_prior = pinv(C_prior)
        
        if np.all(I_prior == 0):
            print("Using uninformative prior (I_prior = 0).")
        else:
            print("Using informative prior (I_prior = C_prior_inv).")
            
        # --- Pre-calculate all Jacobians (NOW IN PARALLEL) ---
        G1 = self._build_jacobian_F1(n1_list, model_params_m_i)
        G2 = self._build_jacobian_F2(n2_list, model_params_m_i)
        G3 = self._build_jacobian_F3(n3_list, model_params_m_i)
        
        # --- Pre-calculate all C_inv ---
        C1_inv = self._build_cov_matrix_inv(num_n1, sigma_F1)
        C2_inv = self._build_cov_matrix_inv(num_n2, sigma_F2)
        C3_inv = self._build_cov_matrix_inv(num_n3, sigma_F3)

        # --- Pre-calculate all Fisher Matrices (I1, I2, I3) ---
        print("\nCalculating individual Fisher matrices (I1, I2, I3)...")
        I1 = self._calculate_fisher_matrix(G1, C1_inv, num_i)
        I2 = self._calculate_fisher_matrix(G2, C2_inv, num_i)
        I3 = self._calculate_fisher_matrix(G3, C3_inv, num_i)
        
        I_matrices = {'1': I1, '2': I2, '3': I3}
        
        # --- Calculate C_post for each combination ---
        results = {}
        print("\nCalculating Posterior Covariance for combinations:")
        
        for combo_name, combo_keys in combinations_dict.items():
            print(f"  Combination: '{combo_name}' (Observables: {combo_keys})")
            I_total_obs = np.zeros((num_i, num_i))
            
            for key in combo_keys:
                if key in I_matrices:
                    I_total_obs += I_matrices[key]
                else:
                    print(f"    Warning: Key '{key}' not in ['1', '2', '3']. Skipping.")
            
            # I_post = I_obs + I_prior
            I_posterior = I_total_obs + I_prior
            
            # C_post = (I_post)^-1
            try:
                C_post = inv(I_posterior)
                results[combo_name] = C_post
            except np.linalg.LinAlgError:
                print(f"    Error: Posterior Information Matrix for '{combo_name}' is singular.")
                print("           This means the parameters are not constrained by this observable set.")
                results[combo_name] = np.full((num_i, num_i), np.inf)

        print("--- All calculations complete ---\n")
        return results

In [7]:
# Cell 5: Main Execution Block

import numpy as np
import os
import warnings
import traceback

# --- 1. USER: Define All Parameters Here ---

# --- 1a. 结果和格式 ---
nmin = 0
nmax = 30
OUTPUT_FOLDER_NAME = "Fisher_Results_Segmented_"+str(nmin)+"n"+str(nmax)+"_1127"
FLOAT_PRECISION = 5 # 输出文件中科学计数法的有效位数

# --- 1b. 模型参数 (m_i) ---
# 定义径向分段 (r_min, r_max)，单位：米
# (程序将自动为每个分段创建 delta_rho, delta_kappa, delta_mu 三个参数)
RADIAL_SEGMENTS = [
    (0, 100e3),
    (100e3, 200e3),
    (200e3, 352e3),
    (353e3, 480e3),
    (481e3, 600e3),
    (600e3, 700e3),
    (700e3, 800e3),
    (800e3, 900e3),
    (900e3, 1000e3),
    (1000e3, 1100e3),
    (1100e3, 1200e3),
    (1200e3, 1300e3),
    (1300e3, 1400e3),
    (1400e3, 1500e3),
    (1500e3, 1600e3),
    (1600e3, 1709e3),
    (1709.1e3, 1725e3), 
    (1725.1e3, 1737.1e3)
]
NUM_SEGMENTS = len(RADIAL_SEGMENTS)

# --- 1c. 文件路径 (与原 Cell 4 相同) ---
# !! PLEASE UPDATE THESE PATHS !!
base_dir = r"D:\Study\Research & Survey\Seismic GW detector\MyWork\Inverse-1D-pert\MultModel_Results1024\Reference_Model"
model_file_path = os.path.join(base_dir, "Reference_Model_8000_code.txt")
eigen_freq_file_s = os.path.join(base_dir, "res1.txt")
eigen_func_folder_s = os.path.join(base_dir, "db25new")

# --- 1d. 模式加载范围 (与原 Cell 4 相同) ---
N_MIN_SPHER = 0
N_MAX_SPHER = 200 # 总共加载 n=0 到 200 的模式
all_s2_n_values_to_load = list(range(N_MIN_SPHER, N_MAX_SPHER + 1))

# --- 1e. 观测量 (与原 Cell 6 相同) ---
# 定义用于 Fisher 矩阵计算的 n 值范围 (必须是 1d 中加载范围的子集)
n_range_F1_freqs = range(nmin, nmax)  # 使用 n=0..149 的频率
n_range_F2_U_amps = range(nmin, nmax) # 使用 n=0..149 的 U 振幅
n_range_F3_V_amps = range(nmin, nmax) # 使用 n=0..149 的 V 振幅

# 观测误差 (相对误差)
sigma_F1 = 1e-4  # 频率
sigma_F2 = 1e-2  # U-振幅
sigma_F3 = 1e-2  # V-振幅

# --- 1f. 观测组合 (与原 Cell 6 相同) ---
combinations_to_run = {
    'F1_(Freqs)_Only': ['1'],
    'F1_plus_F3': ['1', '3'],
    'F1_plus_F2_plus_F3': ['1', '2', '3']
}

# --- 1g. (高级) 辅助函数 ---
def _get_model_value(r1, r2, r_model, model_data):
    """
    获取分段中点的模型值。
    (基于用户的保证：分段内模型参数均一)
    """
    # 查找最接近分段中点的r值的索引
    r_mid = (r1 + r2) / 2.0
    idx = np.argmin(np.abs(r_model - r_mid))
    val = model_data[idx]
    if val == 0:
        print(f"Warning: Model value is 0 for segment ({r1/1e3:.1f}-{r2/1e3:.1f} km).")
        return 1e-30 # 返回一个小值避免除零
    return val

def _sanitize_filename(name):
    """清理组合名称，使其成为有效的文件名"""
    return name.replace(' ', '_').replace('+', 'plus') \
             .replace('(', '').replace(')', '') \
             .replace('/', '').replace('\\', '') + ".txt"


# --- 2. 初始化 (加载数据和 Calculator) ---

print("--- 2. Starting Initialization ---")
try:
    # --- 2a. 创建输出文件夹 ---
    if not os.path.exists(OUTPUT_FOLDER_NAME):
        os.makedirs(OUTPUT_FOLDER_NAME)
        print(f"Created output directory: {OUTPUT_FOLDER_NAME}")
    else:
        print(f"Output directory exists: {OUTPUT_FOLDER_NAME}")

    # --- 2b. 加载频率 (来自原 Cell 4) ---
    full_omega_k2_map_from_file = {}
    spher_nl_tuples_to_load = [(n, 2) for n in all_s2_n_values_to_load]
    _load_frequencies_from_res1(
        res1_path=eigen_freq_file_s, 
        sigma_char='S', 
        target_map=full_omega_k2_map_from_file, 
        nl_tuples_to_consider=spher_nl_tuples_to_load
    )

    # --- 2c. 初始化 SimplifiedS2Calculator (来自原 Cell 4) ---
    my_calculator = SimplifiedS2Calculator(
        all_s2_n_values=all_s2_n_values_to_load,
        model_file=model_file_path,
        eigenfunction_folder_spher=eigen_func_folder_s,
        omega_k2_map=full_omega_k2_map_from_file,
        G_const=0.0 
    )

except Exception as e:
    print(f"\nAn error occurred during SETUP: {e}")
    traceback.print_exc()
    # 如果初始化失败，设置一个空实例
    my_calculator = type('obj', (object,), {'num_k_modes': 0})


# --- 3. 运行 Fisher 矩阵计算 (主要逻辑) ---

if my_calculator.num_k_modes > 0:
    print("\n--- 3. Starting Fisher Matrix Calculation ---")
    try:
        # --- 3a. 生成模型参数列表 (m_i) ---
        model_params_m_i = []
        output_rows_data = [] # 存储用于输出的辅助信息
        
        print(f"Generating {NUM_SEGMENTS * 3} model parameters ({NUM_SEGMENTS} segments x 3 types)...")
        
        for r_min, r_max in RADIAL_SEGMENTS:
            # 存储每行的模型值，用于最后的归一化
            row_info = {
                'r1': r_min, 
                'r2': r_max,
                'model_rho': _get_model_value(r_min, r_max, my_calculator.r_model, my_calculator.rho_model),
                'model_kappa': _get_model_value(r_min, r_max, my_calculator.r_model, my_calculator.kappa_model),
                'model_mu': _get_model_value(r_min, r_max, my_calculator.r_model, my_calculator.mu_model)
            }
            output_rows_data.append(row_info)
            
            # 按 (rho, kappa, mu) 的顺序为每个分段添加参数
            # 这确保了 all_std_devs 向量中的顺序是 [std_rho1, std_kappa1, std_mu1, std_rho2, ...]
            model_params_m_i.append(('delta_rho', (r_min, r_max)))
            model_params_m_i.append(('delta_kappa', (r_min, r_max)))
            model_params_m_i.append(('delta_mu', (r_min, r_max)))
        
        num_i = len(model_params_m_i)
        
        # --- 3b. 设置 Prior 和观测半径 ---
        C_prior = np.zeros((num_i, num_i)) # 无信息先验
        r_obs = my_calculator.R_surface    # 在表面观测
        
        # --- 3c. 初始化 FisherMatrixCalculator ---
        fisher_calc = FisherMatrixCalculator(my_calculator, r_obs_meters=r_obs)
        
        # --- 3d. 运行计算 ---
        all_C_post_results = fisher_calc.calculate_posterior_covariance(
            model_params_m_i=model_params_m_i,
            n_range_F1=n_range_F1_freqs,
            n_range_F2=n_range_F2_U_amps,
            n_range_F3=n_range_F3_V_amps,
            sigma_F1=sigma_F1,
            sigma_F2=sigma_F2,
            sigma_F3=sigma_F3,
            C_prior=C_prior,
            combinations_dict=combinations_to_run
        )
        
        # --- 4. 处理和保存结果 ---
        print("\n--- 4. Processing and Saving Results ---")
        
        float_format_str = f"{{:.{FLOAT_PRECISION}e}}" # e.g., "{:.6e}"
        
        for combo_name, C_post in all_C_post_results.items():
            
            output_filepath = os.path.join(OUTPUT_FOLDER_NAME, _sanitize_filename(combo_name))
            print(f"  Saving results for '{combo_name}' to '{output_filepath}'")
            
            # 提取所有参数的标准差
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                # all_std_devs 是一个长度为 3*x 的向量
                all_std_devs = np.sqrt(np.diag(C_post))
            
            with open(output_filepath, 'w') as f:
                # 写入文件头
                header = (
                    f"#{'r1(km)':<8} {'r2(km)':<8} "
                    f"{'std_rho':<15} {'std_kappa':<15} {'std_mu':<15} "
                    f"{'std_rho/rho':<15} {'std_kappa/kappa':<15} {'std_mu/mu':<15}\n"
                )
                f.write(header)
                
                # 遍历每个分段
                for i in range(NUM_SEGMENTS):
                    # 从 std_devs 向量中解开对应分段的 (rho, kappa, mu) 标准差
                    std_rho   = all_std_devs[i*3 + 0]
                    std_kappa = all_std_devs[i*3 + 1]
                    std_mu    = all_std_devs[i*3 + 2]
                    
                    # 获取该分段的辅助信息 (r1, r2, model_values)
                    seg_info = output_rows_data[i]
                    
                    # 计算归一化标准差
                    norm_std_rho = std_rho / seg_info['model_rho']
                    norm_std_kappa = std_kappa / seg_info['model_kappa']
                    norm_std_mu = std_mu / seg_info['model_mu']
                    
                    # 格式化输出
                    r1_str = f"{seg_info['r1'] / 1e3:<8.1f}"
                    r2_str = f"{seg_info['r2'] / 1e3:<8.1f}"
                    
                    std_rho_str   = f"{float_format_str.format(std_rho):<15}"
                    std_kappa_str = f"{float_format_str.format(std_kappa):<15}"
                    std_mu_str    = f"{float_format_str.format(std_mu):<15}"
                    
                    norm_rho_str   = f"{float_format_str.format(norm_std_rho):<15}"
                    norm_kappa_str = f"{float_format_str.format(norm_std_kappa):<15}"
                    norm_mu_str    = f"{float_format_str.format(norm_std_mu):<15}"
                    
                    # 写入行
                    f.write(
                        f"{r1_str} {r2_str} "
                        f"{std_rho_str} {std_kappa_str} {std_mu_str} "
                        f"{norm_rho_str} {norm_kappa_str} {norm_mu_str}\n"
                    )

        print("\n--- All results saved. ---")
        
    except Exception as e:
        print(f"\nAn error occurred during CALCULATION: {e}")
        traceback.print_exc()

else:
    print("\n'my_calculator' instance not found or is empty.")
    print("Please check initialization steps (Cell 3) and file paths (Cell 5).")

--- 2. Starting Initialization ---
Created output directory: Fisher_Results_Segmented_0n30_1127
Attempting to load S-mode frequencies from: D:\Study\Research & Survey\Seismic GW detector\MyWork\Inverse-1D-pert\MultModel_Results1024\Reference_Model\res1.txt
  Successfully loaded 201 relevant S-mode frequencies.
--- Initializing SimplifiedS2Calculator ---
         This means g_model, p_val, and V_rho kernel will be 0.
Loading background model from: D:\Study\Research & Survey\Seismic GW detector\MyWork\Inverse-1D-pert\MultModel_Results1024\Reference_Model\Reference_Model_8000_code.txt
Model loaded: 8000 radial points, R = 1737.1 km

Found 201 valid (S, n, 2) modes to process.
Preparing all eigenfunction derivatives (S,l=2) (parallelized)...


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 23 concurrent workers.
[Parallel(n_jobs=-2)]: Done  26 out of 201 | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done 197 out of 201 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done 201 out of 201 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.



All 201 (S,l=2) eigenfunctions processed.
--- Initialization Complete ---

--- 3. Starting Fisher Matrix Calculation ---
Generating 54 model parameters (18 segments x 3 types)...
--- Initializing FisherMatrixCalculator ---
Observation radius set to r = 1737.1 km (requested 1737.1 km)
Pre-calculating amplitudes U_k, V_k at observation radius...
Pre-calculating I_k integrals (Integral[ d(mu)/dr * [U_k + C*V_k] * r^2 ] dr)...
--- FisherMatrixCalculator Initialized (201 modes) ---

Calculating for 54 model parameters (m_i).
Observables: 30 (F1-Freq), 30 (F2-U_Amp), 30 (F3-V_Amp)
Using uninformative prior (I_prior = 0).
Building Jacobian G1 (Frequencies) parallelized over 54 parameters...


[Parallel(n_jobs=-1)]: Done 18 out of 54 | elapsed:    4.5s remaining:    9.1s
[Parallel(n_jobs=-1)]: Done 29 out of 54 | elapsed:    7.2s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done 40 out of 54 | elapsed:    9.8s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 51 out of 54 | elapsed:   12.4s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 54 out of 54 | elapsed:   13.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


Building Jacobian G2 (U-Amplitude) parallelized over 54 parameters...


[Parallel(n_jobs=-1)]: Done 18 out of 54 | elapsed:    7.9s remaining:   15.9s
[Parallel(n_jobs=-1)]: Done 29 out of 54 | elapsed:   11.4s remaining:    9.8s
[Parallel(n_jobs=-1)]: Done 40 out of 54 | elapsed:   14.9s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done 51 out of 54 | elapsed:   17.7s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done 54 out of 54 | elapsed:   18.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


Building Jacobian G3 (V-Amplitude) parallelized over 54 parameters...


[Parallel(n_jobs=-1)]: Done 18 out of 54 | elapsed:    7.7s remaining:   15.4s
[Parallel(n_jobs=-1)]: Done 29 out of 54 | elapsed:   10.9s remaining:    9.4s
[Parallel(n_jobs=-1)]: Done 40 out of 54 | elapsed:   14.1s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done 51 out of 54 | elapsed:   17.4s remaining:    0.9s



Calculating individual Fisher matrices (I1, I2, I3)...

Calculating Posterior Covariance for combinations:
  Combination: 'F1_(Freqs)_Only' (Observables: ['1'])
  Combination: 'F1_plus_F3' (Observables: ['1', '3'])
  Combination: 'F1_plus_F2_plus_F3' (Observables: ['1', '2', '3'])
--- All calculations complete ---


--- 4. Processing and Saving Results ---
  Saving results for 'F1_(Freqs)_Only' to 'Fisher_Results_Segmented_0n30_1127\F1_Freqs_Only.txt'
  Saving results for 'F1_plus_F3' to 'Fisher_Results_Segmented_0n30_1127\F1_plus_F3.txt'
  Saving results for 'F1_plus_F2_plus_F3' to 'Fisher_Results_Segmented_0n30_1127\F1_plus_F2_plus_F3.txt'

--- All results saved. ---


[Parallel(n_jobs=-1)]: Done 54 out of 54 | elapsed:   17.8s finished
